# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f36f683c580>
├── 'a' --> tensor([[ 0.1221, -0.0711, -0.8307],
│                   [-0.4032, -0.6506,  0.9138]])
└── 'x' --> <FastTreeValue 0x7f36f683c940>
    └── 'c' --> tensor([[ 1.4819,  1.0881,  0.2656,  1.4813],
                        [-0.2772,  0.8231, -1.4465, -0.6323],
                        [ 0.3639,  0.7173, -0.4586,  1.9242]])

In [4]:
t.a

tensor([[ 0.1221, -0.0711, -0.8307],
        [-0.4032, -0.6506,  0.9138]])

In [5]:
%timeit t.a

87.3 ns ± 0.12 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f36f683c580>
├── 'a' --> tensor([[ 0.6257,  0.5916, -0.9458],
│                   [ 0.9168,  1.6415,  0.4957]])
└── 'x' --> <FastTreeValue 0x7f36f683c940>
    └── 'c' --> tensor([[ 1.4819,  1.0881,  0.2656,  1.4813],
                        [-0.2772,  0.8231, -1.4465, -0.6323],
                        [ 0.3639,  0.7173, -0.4586,  1.9242]])

In [7]:
%timeit t.a = new_value

92 ns ± 0.144 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.1221, -0.0711, -0.8307],
               [-0.4032, -0.6506,  0.9138]]),
    x: Batch(
           c: tensor([[ 1.4819,  1.0881,  0.2656,  1.4813],
                      [-0.2772,  0.8231, -1.4465, -0.6323],
                      [ 0.3639,  0.7173, -0.4586,  1.9242]]),
       ),
)

In [10]:
b.a

tensor([[ 0.1221, -0.0711, -0.8307],
        [-0.4032, -0.6506,  0.9138]])

In [11]:
%timeit b.a

77.5 ns ± 0.0745 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 2.2391,  0.1586, -0.9907],
               [ 0.0313,  0.0274, -0.1984]]),
    x: Batch(
           c: tensor([[ 1.4819,  1.0881,  0.2656,  1.4813],
                      [-0.2772,  0.8231, -1.4465, -0.6323],
                      [ 0.3639,  0.7173, -0.4586,  1.9242]]),
       ),
)

In [13]:
%timeit b.a = new_value

647 ns ± 0.818 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

1.26 µs ± 5.34 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

12.6 µs ± 48.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

171 µs ± 8.86 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

168 µs ± 491 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f36f47c2400>
├── 'a' --> tensor([[[ 0.1221, -0.0711, -0.8307],
│                    [-0.4032, -0.6506,  0.9138]],
│           
│                   [[ 0.1221, -0.0711, -0.8307],
│                    [-0.4032, -0.6506,  0.9138]],
│           
│                   [[ 0.1221, -0.0711, -0.8307],
│                    [-0.4032, -0.6506,  0.9138]],
│           
│                   [[ 0.1221, -0.0711, -0.8307],
│                    [-0.4032, -0.6506,  0.9138]],
│           
│                   [[ 0.1221, -0.0711, -0.8307],
│                    [-0.4032, -0.6506,  0.9138]],
│           
│                   [[ 0.1221, -0.0711, -0.8307],
│                    [-0.4032, -0.6506,  0.9138]],
│           
│                   [[ 0.1221, -0.0711, -0.8307],
│                    [-0.4032, -0.6506,  0.9138]],
│           
│                   [[ 0.1221, -0.0711, -0.8307],
│                    [-0.4032, -0.6506,  0.9138]]])
└── 'x' --> <FastTreeValue 0x7f362eb7b970>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

39.4 µs ± 108 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f36f478c1f0>
├── 'a' --> tensor([[ 0.1221, -0.0711, -0.8307],
│                   [-0.4032, -0.6506,  0.9138],
│                   [ 0.1221, -0.0711, -0.8307],
│                   [-0.4032, -0.6506,  0.9138],
│                   [ 0.1221, -0.0711, -0.8307],
│                   [-0.4032, -0.6506,  0.9138],
│                   [ 0.1221, -0.0711, -0.8307],
│                   [-0.4032, -0.6506,  0.9138],
│                   [ 0.1221, -0.0711, -0.8307],
│                   [-0.4032, -0.6506,  0.9138],
│                   [ 0.1221, -0.0711, -0.8307],
│                   [-0.4032, -0.6506,  0.9138],
│                   [ 0.1221, -0.0711, -0.8307],
│                   [-0.4032, -0.6506,  0.9138],
│                   [ 0.1221, -0.0711, -0.8307],
│                   [-0.4032, -0.6506,  0.9138]])
└── 'x' --> <FastTreeValue 0x7f3626980f40>
    └── 'c' --> tensor([[ 1.4819,  1.0881,  0.2656,  1.4813],
                        [-0.2772,  0.8231, -1.4465, -0.6323],
                 

In [23]:
%timeit t_cat(trees)

36.8 µs ± 50.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

79.2 µs ± 572 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.1221, -0.0711, -0.8307],
                [-0.4032, -0.6506,  0.9138]],
       
               [[ 0.1221, -0.0711, -0.8307],
                [-0.4032, -0.6506,  0.9138]],
       
               [[ 0.1221, -0.0711, -0.8307],
                [-0.4032, -0.6506,  0.9138]],
       
               [[ 0.1221, -0.0711, -0.8307],
                [-0.4032, -0.6506,  0.9138]],
       
               [[ 0.1221, -0.0711, -0.8307],
                [-0.4032, -0.6506,  0.9138]],
       
               [[ 0.1221, -0.0711, -0.8307],
                [-0.4032, -0.6506,  0.9138]],
       
               [[ 0.1221, -0.0711, -0.8307],
                [-0.4032, -0.6506,  0.9138]],
       
               [[ 0.1221, -0.0711, -0.8307],
                [-0.4032, -0.6506,  0.9138]]]),
    x: Batch(
           c: tensor([[[ 1.4819,  1.0881,  0.2656,  1.4813],
                       [-0.2772,  0.8231, -1.4465, -0.6323],
                       [ 0.3639,  0.7173, -0.4586,  1.9242]],
         

In [26]:
%timeit Batch.stack(batches)

97.9 µs ± 171 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.1221, -0.0711, -0.8307],
               [-0.4032, -0.6506,  0.9138],
               [ 0.1221, -0.0711, -0.8307],
               [-0.4032, -0.6506,  0.9138],
               [ 0.1221, -0.0711, -0.8307],
               [-0.4032, -0.6506,  0.9138],
               [ 0.1221, -0.0711, -0.8307],
               [-0.4032, -0.6506,  0.9138],
               [ 0.1221, -0.0711, -0.8307],
               [-0.4032, -0.6506,  0.9138],
               [ 0.1221, -0.0711, -0.8307],
               [-0.4032, -0.6506,  0.9138],
               [ 0.1221, -0.0711, -0.8307],
               [-0.4032, -0.6506,  0.9138],
               [ 0.1221, -0.0711, -0.8307],
               [-0.4032, -0.6506,  0.9138]]),
    x: Batch(
           c: tensor([[ 1.4819,  1.0881,  0.2656,  1.4813],
                      [-0.2772,  0.8231, -1.4465, -0.6323],
                      [ 0.3639,  0.7173, -0.4586,  1.9242],
                      [ 1.4819,  1.0881,  0.2656,  1.4813],
                      [-0.2772,  

In [28]:
%timeit Batch.cat(batches)

182 µs ± 976 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

407 µs ± 6.04 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
